In [1]:
#Загружаю нужные библиотеки

import pandas as pd
import math
import re

# для конвертации в словарь
import ast

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
work = pd.read_excel('C:/00_Data/Стоматологии/Clinic_Denta/clinic_Denta_info.xlsx', sheet_name='zoon')

In [3]:
work.sample(3)

,ID_1dent,ident_method,page_url,object_name,address,region,city,street_type,street,house,...,clinic_rating,review_number,description,clinic_info,doc_pics,specialists_dict,price,services,contacts,soc_links
29,4631,phone,https://zoon.ru/msk/medical/stomatologicheskay...,АМК-Dental,"м. Проспект Вернадского, проспект Вернадского,...",Москва,Москва,проспект,"проспект Вернадского, 61к1","проспект Вернадского, 61к1",...,3.7,0,Здоровье полости рта стоит доверять только ква...,none,0,{'Артур Мурадович Кадыраджиев ': ['Стаж 6 лет'...,"{'Лечение кариеса': '4000—5000', 'Удаление зуб...",{'Тип мед. учреждения': 'стоматологическая кли...,"{'Время работы': ['ежедневно: 09:00 - 21:00', ...",{'Instagram': 'https://zoon.ru/redirect/?to=ht...
15,3031,phone,https://zoon.ru/msk/medical/klinika_stomatolog...,Клиника стоматологии и имплантации SWORD,"м. Шипиловская, Шипиловская улица, 50к3, Москва",Москва,Москва,улица,Шипиловская 50к3,Шипиловская 50к3,...,4.4,6,Стоматологическая клиника располагается на ули...,"['Юридическое название: ООО ""Кумай-2000""', 'Гл...",0,{'Ирина Анатольевна Мечникова ': ['Стоматолог'...,"{'Лечение кариеса': '400—500', 'Удаление зубов...",{'Тип мед. учреждения': 'стоматологическая кли...,"{'Время работы': ['ежедневно: 09:00 - 20:00'],...",none
31,4659,address,https://zoon.ru/msk/medical/stomatologiya_dent...,ДентоСпас,"м. Алтуфьево, Шенкурский проезд, 3Б, 3 этаж, М...",Москва,Москва,проезд,Шенкурский,д.3Б,...,4.6,61,Стоматология «ДентоСпас» оказывает качественны...,"['Юридическое название: ООО ""АлкоСпас""']",0,{'Белла Михайловна Амбарцумян ': ['Стаж 13 лет...,"{'Лечение кариеса': '5000—11000', 'Удаление зу...",{'Тип мед. учреждения': 'стоматологическая кли...,{'Время работы': ['Пн-Сб: 09:00—21:00; Вс: 09:...,none


In [4]:
work.columns

Index(['ID_1dent', 'ident_method', 'page_url', 'object_name', 'address',
       'region', 'city', 'street_type', 'street', 'house', 'phone', 'hours',
       'website', 'social_network', 'fb_link', 'inst_link', 'vk_link',
       'tw_link', 'clinic_type', 'stom_services', 'specialists',
       'clinic_rating', 'review_number', 'description', 'clinic_info',
       'doc_pics', 'specialists_dict', 'price', 'services', 'contacts',
       'soc_links'],
      dtype='object')

In [ ]:
work.shape

In [ ]:
%%time

# вытаскиваю данные из словаря в таблицу

data = [] # список для данных

for i in range(work.shape[0]):
    ID_1dent = work.iloc[[i]]['ID_1dent'].values[0] 
    clinic = work.iloc[[i]]['clinic'].values[0]
    
    doctors_dict = ast.literal_eval(work.iloc[i].specialist)
    
    for doctor in doctors_dict:
        short_name = doctors_dict[doctor]['фио']
        full_name = doctors_dict[doctor]['имя']
        sex = doctors_dict[doctor]['пол']
        experience = doctors_dict[doctor]['стаж']
        specialization = doctors_dict[doctor]['профиль'].lower()
        review_number = doctors_dict[doctor]['кол-во отзывов']
        
        data.append([ID_1dent, clinic, short_name, full_name, sex, experience, specialization, review_number])   

In [ ]:
len(data)

In [ ]:
head = ['ID_1dent', 'clinic', 'short_name', 'full_name', 'sex', 'experience', 'specialization', 'review_number']
work_doctors = pd.DataFrame(data, columns=head)

In [ ]:
work_doctors.sample(3)

In [ ]:
work_doctors.shape

In [ ]:
work_doctors.dtypes

In [ ]:
# проверка на пустые значения

work_doctors.isnull().sum()

In [ ]:
work_doctors['review_number'] = work.review_number.replace(r'\s.+', '')

In [ ]:
work_doctors.review_number.fillna('0', inplace=True)

In [ ]:
work_doctors.experience.fillna('0', inplace=True)

In [ ]:
# меняю типы столбцов

work_doctors = work_doctors.astype({'ID_1dent': 'int32', 'experience': 'int32', 'review_number': 'int32'})

### Средний стаж ###

In [ ]:
# средний стаж врачей в клинике

doctor_avr_experience = (work_doctors[work_doctors.experience > 0]
                         .groupby(['ID_1dent', 'clinic'], as_index=False)
                         .agg({'experience': 'mean'}).round(1)
                         .rename(columns={'experience': 'avr_experience'})
                        )

In [ ]:
doctor_avr_experience.sample(3)

In [ ]:
work = work.merge(doctor_avr_experience, how='left', on=['ID_1dent', 'clinic'])

In [ ]:
work.shape

### Специализация клиник ###

In [ ]:
# составляю список уникальных специальностей

doc_specialization = [] # список для данных

for i in work_doctors['specialization']: # вытаскиваю циклом
    if i != '': # fillna не помог
        doc_specialization += i.split(', ')
    
doc_specialization = list(set(doc_specialization)) # удаляю повторы

In [ ]:
# сортирую

doc_specialization.sort()

doc_specialization

In [ ]:

doc_specialization_dict = {} # словарь для специализаций

for specialty in doc_specialization:
    doc_number = 0
    
    for item in work_doctors['specialization'].values:
        for i in item.split(', '):
            doc_number += i.count(specialty)
                    
    doc_specialization_dict[specialty] = doc_number

In [ ]:
doc_specialization_dict

In [ ]:
header = ['specialty', 'doc_number']
work_doc_specialization = pd.DataFrame(list(doc_specialization_dict.items()), columns=header)

In [ ]:
work_doc_specialization

### Вытаскиваю специализации врачей в плоскую таблицу ###

In [ ]:
# вытаскиваю специализации 

for specialty in doc_specialization:
    temp_list = []
    
    for i in range(work_doctors.shape[0]):
        temp_list.append(work_doctors.loc[i].specialization.split(', ').count(specialty))
        
    work_doctors[specialty] = temp_list

In [ ]:
work_doctors.head()

In [ ]:
work_doctors.to_excel('C:/00_Data/Стоматологии/Clinic_Denta/temp.xlsx')

In [ ]:
doc_specialization

In [ ]:
# список с нужными специальностями

doc_specialization_need = ['гнатолог',
                           'детский стоматолог',
                           'пародонтолог',
                           'стоматолог',
                           'стоматолог-гигиенист',
                           'стоматолог-имплантолог',
                           'стоматолог-ортодонт',
                           'стоматолог-ортопед',
                           'стоматолог-хирург',
                           'терапевт',
                           'челюстно-лицевой хирург']

In [ ]:
clinic_specialization = (work_doctors
                         .groupby('clinic', as_index=False)
                         .agg({'гнатолог': 'sum',
                               'детский стоматолог': 'sum',
                               'пародонтолог': 'sum',
                               'стоматолог': 'sum',
                               'стоматолог-гигиенист': 'sum',
                               'стоматолог-имплантолог': 'sum',
                               'стоматолог-ортодонт': 'sum',
                               'стоматолог-ортопед': 'sum',
                               'стоматолог-хирург': 'sum',
                               'терапевт': 'sum',
                               'челюстно-лицевой хирург': 'sum'})
                        )

In [ ]:
clinic_specialization['all_specialties'] = clinic_specialization.iloc[:, 1:].sum(axis=1)

In [ ]:
clinic_specialization.sample()

In [ ]:
clinic_specialization['pediatric_%'] = round(
    clinic_specialization['детский стоматолог'] / clinic_specialization['all_specialties'],
    2)

In [ ]:
clinic_specialization['orthodontist_%'] = round(
    clinic_specialization['стоматолог-ортодонт'] / clinic_specialization['all_specialties'],
    2)

In [ ]:
work.sample()

In [ ]:
work = work.merge(clinic_specialization, on='clinic', how='left')

In [ ]:
work.to_excel('C:/00_Data/Стоматологии/Clinic_Denta/temp1.xlsx')